In [77]:
import os
import json
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split

import sagemaker
from sagemaker import clarify
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

In [7]:
# Set region, boto3 and SageMaker SDK variables¶

#You can change this to a region of your choice
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity()["Account"]

random_state = 42

Using AWS Region: us-east-1


In [8]:
%store
%store -r

Stored variables and their in-db values:
clarify_data_uri                -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
data_prefix                     -> 'sagemaker-tutorial/data'
default_bucket                  -> 'sagemaker-us-east-1-367158743199'
feature_group_name              -> 'FG-flow-sm-tutorial-31-16-16-17-9f41d66b'
hyperparameters                 -> {'max_depth': '5', 'eta': '0.2', 'gamma': '4', 'mi
model_data                      -> 's3://sagemaker-us-east-1-367158743199/tf2-resnet-
prefix                          -> 'sagemaker-tutorial'
s3_raw_data                     -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
s3_train_data                   -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
s3_validation_data              -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
test_data_uri                   -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
train_data_uri                  -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
traini

## Get the data from offline feature store

Feature Store provides offline storage for feature values in your S3 bucket. Your data is stored in your S3 bucket using a prefixing scheme based on event time. The offline store is an append-only store, enabling Feature Store to maintain a historical record of all feature values. Data is stored in the offline store in Parquet format for optimized storage and query access.

You can query, explore, and visualize features using Data Wrangler from Amazon SageMaker Studio.  Feature Store supports combining data to produce, train, validate, and test data sets, and allows you to extract data at different points in time.

 
<span style="color:red">**TODO:  THE CODE NEEDS TO CHANGE**</span>


In [9]:
# featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', 
#                                            region_name=region
#                                           )

# feature_store_session = Session(
#     boto_session=boto_session,
#     sagemaker_client=sagemaker_boto_client,
#     sagemaker_featurestore_runtime_client=featurestore_runtime
# )

# offline_feature_store_bucket = f's3://{default_bucket}/{account_id}/sagemaker/{region}/offline-store/{feature_group_name}/data/year=2021/month=03/day=31/hour=16/'
# offline_feature_store_bucket

# !aws s3 cp $offline_feature_store_bucket ../sm-tutorial/02_build_train/ --recursive

# # offline_feature_store_bucket = f's3://{default_bucket}/'
# fg_prefix = f'sagemaker/{region}/offline-store/{feature_group_name}/data/'
# s3_client.list_objects_v2(Bucket=default_bucket,
#                          Prefix=fg_prefix,
#                          Delimiter='/')

# def download_all_objects_in_folder():
#     s3_resource = boto3.resource('s3')
#     my_bucket = s3_resource.Bucket(default_bucket)
#     objects = my_bucket.objects.filter(Prefix=offline_feature_prefix)
#     for obj in objects:
#         path, filename = os.path.split(obj.key)
#         my_bucket.download_file(obj.key, filename)

In [16]:
file_names = ['20210331T162204Z_ATxVKv9V8rL9hJyQ.parquet',
             '20210331T162204Z_BVs0QiuqNaVyrXTY.parquet',
             '20210331T162204Z_KnPgBMRO3yEo3BP3.parquet	']

local_processed_data = '../sm-tutorial/02_build_train/processed_data/parquet/'
for f in file_names:
    s3_path = f's3://sagemaker-us-east-1-367158743199/367158743199/sagemaker/us-east-1/offline-store/FG-flow-sm-tutorial-31-16-16-17-9f41d66b-1617207382/data/year=2021/month=03/day=31/hour=16/{f}'

    ! aws s3 cp $s3_path $local_processed_data

download: s3://sagemaker-us-east-1-367158743199/367158743199/sagemaker/us-east-1/offline-store/FG-flow-sm-tutorial-31-16-16-17-9f41d66b-1617207382/data/year=2021/month=03/day=31/hour=16/20210331T162204Z_ATxVKv9V8rL9hJyQ.parquet to ../sm-tutorial/02_build_train/processed_data/parquet/20210331T162204Z_ATxVKv9V8rL9hJyQ.parquet
download: s3://sagemaker-us-east-1-367158743199/367158743199/sagemaker/us-east-1/offline-store/FG-flow-sm-tutorial-31-16-16-17-9f41d66b-1617207382/data/year=2021/month=03/day=31/hour=16/20210331T162204Z_BVs0QiuqNaVyrXTY.parquet to ../sm-tutorial/02_build_train/processed_data/parquet/20210331T162204Z_BVs0QiuqNaVyrXTY.parquet
download: s3://sagemaker-us-east-1-367158743199/367158743199/sagemaker/us-east-1/offline-store/FG-flow-sm-tutorial-31-16-16-17-9f41d66b-1617207382/data/year=2021/month=03/day=31/hour=16/20210331T162204Z_KnPgBMRO3yEo3BP3.parquet to ../sm-tutorial/02_build_train/processed_data/parquet/20210331T162204Z_KnPgBMRO3yEo3BP3.parquet


In [17]:
local_processed_data

'../sm-tutorial/02_build_train/processed_data/parquet/'

In [18]:
import pyarrow.parquet as pq

def join_parquet_files(dir_path=local_processed_data):
    all_files = os.listdir(dir_path)


    df = pd.DataFrame()
    for f in all_files:
        full_path = os.path.join(dir_path, f)
        df_partial = pq.read_table(full_path).to_pandas()
        df = pd.concat([df, df_partial], axis=0)
    
    return df

In [19]:
df_processed = join_parquet_files()

## Split DataFrame into Train & Test Sets

In [20]:
# drop feature store created columns
df_processed = df_processed.drop(columns=['ID', 'EVENT_TIME', 'write_time', 'api_invocation_time', 'is_deleted'])

In [27]:
df_processed['LABEL'] = df_processed['LABEL'].astype('str')

In [28]:
X_train, X_test = train_test_split(df_processed, test_size=0.2, random_state=random_state)

X_train, X_val = train_test_split(X_test, test_size=0.2, random_state=random_state)

In [29]:
X_train.to_csv(f'{local_processed_data}/train.csv', header=False, index=False)

response = sagemaker_session.upload_data(f'{local_processed_data}/train.csv',
                                         bucket=default_bucket, 
                                         key_prefix=data_prefix)
train_data_uri = response
%store train_data_uri

Stored 'train_data_uri' (str)


In [30]:
# # save the same training file with header for clarify
# X_train.to_csv(f'{local_processed_data}/train.csv', header=True, index=False)

# response = sagemaker_session.upload_data(f'{local_processed_data}/train.csv',
#                                          bucket=default_bucket, 
#                                          key_prefix=data_prefix)
# clarify_data_uri = response
# %store clarify_data_uri

In [31]:
X_val.to_csv(f'{local_processed_data}/validation.csv', header=False, index=False)

response = sagemaker_session.upload_data(f'{local_processed_data}/validation.csv',
                                         bucket=default_bucket, 
                                         key_prefix=data_prefix)
validation_data_uri = response
%store validation_data_uri

Stored 'validation_data_uri' (str)


In [32]:
X_test.to_csv(f'{local_processed_data}/test.csv', header=False, index=False)

response = sagemaker_session.upload_data(f'{local_processed_data}/test.csv',
                                         bucket=default_bucket, 
                                         key_prefix=data_prefix)
test_data_uri = response
%store test_data_uri

Stored 'test_data_uri' (str)


# Train a model using XGBoost

 
<span style="color:red">**TODO:  XGBoost details**</span>


## Set the hyperparameters

 
<span style="color:red">**TODO:  XGBoost hyperparameters details**</span>


In [33]:
hyperparameters = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
    "num_round": "50"}

%store hyperparameters

Stored 'hyperparameters' (dict)


## Create and fit the estimator

In [34]:
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
content_type = "text/csv"
estimator_output_path = f's3://{default_bucket}/{prefix}/training_jobs'

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")


# construct a SageMaker estimator that calls the xgboost-container
xgb_estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container,
                                              hyperparameters=hyperparameters,
                                              role=sagemaker.get_execution_role(),
                                              instance_count=train_instance_count,
                                              instance_type=train_instance_type,
                                              volume_size=5,  # 5 GB
                                              output_path=estimator_output_path)

In [35]:
# define the data type and paths to the training and validation datasets
train_input = TrainingInput(train_data_uri, content_type="text/csv")
validation_input = TrainingInput(validation_data_uri, content_type="text/csv")

# execute the XGBoost training job
xgb_estimator.fit({'train': train_input,
#                    'validation': validation_input
                  }
                   )

training_job_name = xgb_estimator.latest_training_job.job_name
%store training_job_name

2021-04-01 19:55:41 Starting - Starting the training job...
2021-04-01 19:56:04 Starting - Launching requested ML instancesProfilerReport-1617306941: InProgress
......
2021-04-01 19:57:04 Starting - Preparing the instances for training.........
2021-04-01 19:58:35 Downloading - Downloading input data
2021-04-01 19:58:35 Training - Downloading the training image.....[2021-04-01 19:59:20.090 ip-10-0-235-37.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
INFO:root:Train matrix 

Here we create the SageMaker model.

In [36]:
training_job_name

'sagemaker-xgboost-2021-04-01-19-55-41-144'

In [37]:
model_name = f'{training_job_name}-model'
model = xgb_estimator.create_model(name=model_name)
container_def = model.prepare_container_def()

sagemaker_session.create_model(model_name,
                     sagemaker_role,
                     container_def)

'sagemaker-xgboost-2021-04-01-19-55-41-144-model'

# Amazon SageMaker Clarify
Now that you have your model set up. Let’s say hello to SageMaker Clarify!



In [38]:
clarify_processor = clarify.SageMakerClarifyProcessor(role=sagemaker_role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

## Detecting Bias

SageMaker Clarify helps you detect possible pre- and post-training biases using a variety of metrics. 

A `DataConfig` object communicates some basic information about data I/O to SageMaker Clarify. We specify where to find the input dataset, where to store the output, the target column (`label`), the header names, and the dataset type.

In [39]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(default_bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_data_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='LABEL',
                                      headers=df_processed.columns.to_list(),
                                      dataset_type='text/csv')

A `ModelConfig` object communicates information about your trained model. To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. * instance_type and instance_count specify your preferred instance type and instance count used to run your model on during SageMaker Clarify’s processing. The testing dataset is small so a single standard instance is good enough to run this example. If your have a large complex dataset, you may want to use a better instance type to speed up, or add more instances to enable Spark parallelization. * accept_type denotes the endpoint response payload format, and content_type denotes the payload format of request to the endpoint.

In [40]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

A `ModelPredictedLabelConfig` provides information on the format of your predictions. XGBoost model outputs probabilities of samples, so SageMaker Clarify invokes the endpoint then uses `probability_threshold` to convert the probability to binary labels for bias analysis. Prediction above the threshold is interpreted as label value 1 and below or equal as label value 0.

In [41]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

### Writing BiasConfig

SageMaker Clarify also needs information on what the sensitive columns (`facets`) are, what the sensitive features (`facet_values_or_threshold`) may be, and what the desirable outcomes are (`label_values_or_threshold`). SageMaker Clarify can handle both categorical and continuous data for `facet_values_or_threshold` and for `label_values_or_threshold`. In this case we are using categorical data.



We specify this information in the BiasConfig API. Here we use `SEX` as the sensitive group.

group_name is used to form subgroups for the measurement of Conditional Demographic Disparity in Labels (CDDL) and Conditional Demographic Disparity in Predicted Labels (CDDPL) with regards to Simpson’s paradox.

In [42]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[1],
                                facet_name='SEX',
                                facet_values_or_threshold=[0],
                                group_name='AGE'
                                )

 
<span style="color:red">**TODO:  PRETRIANING BIAS WITH WRANGLER**</span>


refer to smote

## Post-training Bias

Computing post-training bias metrics does require a trained model.

Unbiased training data (as determined by concepts of fairness measured by bias metric) may still result in biased model predictions after training. Whether this occurs depends on several factors including hyperparameter choices.

You can run these options separately with `run_pre_training_bias()` and `run_post_training_bias()` or at the same time with `run_bias()` as shown below.


In [43]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')


Job Name:  Clarify-Bias-2021-04-01-19-59-54-916
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-367158743199/sagemaker-tutorial/data/train.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-367158743199/Clarify-Bias-2021-04-01-19-59-54-916/input/analysis_config/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-367158743199/sagemaker-tutorial/clarify-bias', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..........................

UnexpectedStatusException: Error for Processing job Clarify-Bias-2021-04-01-19-59-54-916: Failed. Reason: ClientError: Predicted Label Column series datatype is not the same as Label Column series


<span style="color:red">**TODO:  SCREENSHOTS OF BIAS REPORT IN STUDIO**</span>

https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.html


# Model Lineage

Amazon SageMaker ML Lineage Tracking creates and stores information about the steps of a machine learning (ML) workflow from data preparation to model deployment. With the tracking information you can reproduce the workflow steps, track model and dataset lineage, and establish model governance and audit standards. With SageMaker Lineage Tracking data scientists and model builders can do the following:

* Keep a running history of model discovery experiments.
* Establish model governance by tracking model lineage artifacts for auditing and compliance verification.
* Clone and rerun workflows to experiment with what-if scenarios while developing models.
* Share a workflow that colleagues can reproduce and enhance (for example, while collaborating on solving a business problem).
* Clone and rerun workflows with additional debugging or logging routines, or new input variations for troubleshooting issues in production models.

In [48]:
training_job_info = sagemaker_boto_client.describe_training_job(TrainingJobName=training_job_name)

#### Training data artifact

In [59]:
training_data_s3_uri = training_job_info['InputDataConfig'][0]['DataSource']['S3DataSource']['S3Uri']

matching_artifacts = list(sagemaker.lineage.artifact.Artifact.list(
    source_uri=training_data_s3_uri,
    sagemaker_session=sagemaker_session))

if matching_artifacts:
    training_data_artifact = matching_artifacts[0]
    print(f'Using existing artifact: {training_data_artifact.artifact_arn}')
else:
    training_data_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name='TrainingData',
        source_uri=training_data_s3_uri,
        artifact_type='Dataset',
        sagemaker_session=sagemaker_session)
    print(f'Create artifact {training_data_artifact.artifact_arn}: SUCCESSFUL')

Using existing artifact: arn:aws:sagemaker:us-east-1:367158743199:artifact/46e81ac5bb720131c95259d4f2325499


#### Model artifact

In [60]:
trained_model_s3_uri = training_job_info['ModelArtifacts']['S3ModelArtifacts']

matching_artifacts = list(sagemaker.lineage.artifact.Artifact.list(
    source_uri=trained_model_s3_uri,
    sagemaker_session=sagemaker_session))

if matching_artifacts:
    model_artifact = matching_artifacts[0]
    print(f'Using existing artifact: {model_artifact.artifact_arn}')
else:
    model_artifact = sagemaker.lineage.artifact.Artifact.create(
        artifact_name='TrainedModel',
        source_uri=trained_model_s3_uri,
        artifact_type='Model',
        sagemaker_session=sagemaker_session)
    print(f'Create artifact {model_artifact.artifact_arn}: SUCCESSFUL')

Using existing artifact: arn:aws:sagemaker:us-east-1:367158743199:artifact/b1877c142f08be8fbd8abe259c0051c1


### Set artifact associations

In [62]:
trial_component = sagemaker_boto_client.describe_trial_component(TrialComponentName=training_job_1_name+'-aws-training-job')
trial_component_arn = trial_component['TrialComponentArn']

In [63]:
# input artifacts
input_artifacts = [training_data_artifact]

for a in input_artifacts:
    try:
        sagemaker.lineage.association.Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type='ContributedTo',
            sagemaker_session=sagemaker_session)
        print(f"Association with {a.artifact_type}: SUCCEESFUL")
    except:
        print(f"Association already exists with {a.artifact_type}")

Association already exists with DataSet


In [64]:
# output artifacts
output_artifacts = [model_artifact]

for a in output_artifacts:
    try:
        sagemaker.lineage.association.Association.create(
            source_arn=a.artifact_arn,
            destination_arn=trial_component_arn,
            association_type='Produced',
            sagemaker_session=sagemaker_session)
        print(f"Association with {a.artifact_type}: SUCCESSFUL")
    except:
        print(f"Association already exists with {a.artifact_type}")

Association with Model: SUCCESSFUL


## Deposit Model and Lineage in SageMaker Model Registry


### Create Model Package Group

A Model Package Groups holds multiple versions or iterations of a model. Though it is not required to create them for every model in the registry, they help organize various models which all have the same purpose and provide autiomatic versioning.

In [66]:
mpg_name = prefix
%store mpg_name
print(f'Model Package Group name: {mpg_name}')


Stored 'mpg_name' (str)
Model Package Group name: sagemaker-tutorial


In [68]:
mpg_input_dict = {
    'ModelPackageGroupName': mpg_name,
    'ModelPackageGroupDescription': 'Credit Defualt Prediction'
}

In [69]:
matching_mpg = sagemaker_boto_client.list_model_package_groups(NameContains=mpg_name)['ModelPackageGroupSummaryList']

if matching_mpg:
    print(f'Using existing Model Package Group: {mpg_name}')
else:
    mpg_response = sagemaker_boto_client.create_model_package_group(**mpg_input_dict)
    print(f'Create Model Package Group {mpg_name}: SUCCESSFUL')
    %store mpg_name

Create Model Package Group sagemaker-tutorial: SUCCESSFUL
Stored 'mpg_name' (str)


#### Create and upload a metrics report

In [74]:
model_metrics_report = {'classification_metrics': {}}
for metric in training_job_info['FinalMetricDataList']:
    stat = {metric['MetricName']: {'value': metric['Value']}}
    model_metrics_report['classification_metrics'].update(stat)
    
with open('training_metrics.json', 'w') as f:
    json.dump(model_metrics_report, f)
    
metrics_s3_key = f"{prefix}/training_jobs/{training_job_info['TrainingJobName']}/training_metrics.json"
s3_client.upload_file(Filename='training_metrics.json', Bucket=default_bucket, Key=metrics_s3_key)

#### Define model metrics

In [79]:
model_metrics = {
    'ModelQuality': {
        'Statistics': {
            'ContentType': 'application/json',
            'S3Uri': f's3://{default_bucket}/{prefix}/{metrics_s3_key}'
        }
    },
    'Bias': {
        'Report': {
            'ContentType': 'application/json',
            'S3Uri': f'{bias_report_output_path}/analysis.json'
        }
    }
}

#### Define the inference spec

In [91]:
inference_spec ={    
    "InferenceSpecification": {
        "Containers" : [{
            "Image": training_job_info['AlgorithmSpecification']['TrainingImage'],
            "ModelDataUrl": training_job_info['ModelArtifacts']['S3ModelArtifacts']
        }],
        "SupportedTransformInstanceTypes": ["ml.m4.xlarge"],
        "SupportedRealtimeInferenceInstanceTypes": ["ml.m4.xlarge"],
        "SupportedContentTypes": ['text/csv'],
        "SupportedResponseMIMETypes": ['text/csv']
    }
}



# {'ModelDataUrl': }

In [92]:
mp_input_dict = {
    'ModelPackageGroupName': mpg_name,
    'ModelPackageDescription': 'XGBoost classifier to detect insurance fraud.',
    'ModelApprovalStatus': 'PendingManualApproval',
    'ModelMetrics': model_metrics
}

mp_input_dict.update(inference_spec)
mp1_response = sagemaker_boto_client.create_model_package(**mp_input_dict)

In [93]:
mp_info = sagemaker_boto_client.describe_model_package(ModelPackageName=mp1_response['ModelPackageArn'])
mp_status = mp_info['ModelPackageStatus']

while mp_status not in ['Completed', 'Failed']:
    time.sleep(5)
    mp_info = sagemaker_boto_client.describe_model_package(ModelPackageName=mp1_response['ModelPackageArn'])
    mp_status = mp_info['ModelPackageStatus']
    print(f'model package status: {mp_status}')
print(f'model package status: {mp_status}')

model package status: Completed


In [94]:
# view model package in registry
sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList']

[{'ModelPackageGroupName': 'sagemaker-tutorial',
  'ModelPackageVersion': 1,
  'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:367158743199:model-package/sagemaker-tutorial/1',
  'ModelPackageDescription': 'XGBoost classifier to detect insurance fraud.',
  'CreationTime': datetime.datetime(2021, 4, 1, 20, 59, 31, 765000, tzinfo=tzlocal()),
  'ModelPackageStatus': 'Completed',
  'ModelApprovalStatus': 'PendingManualApproval'}]